# 处理 genotype文件

In [1]:
%pylab inline
%matplotlib inline
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
genotype = pd.read_table("data/genotype.dat",sep="\\s+");

## 1. 编码

In [3]:
def ATCG2num(c):
    if c=='A' : return 0
    elif c=='T' : return 1
    elif c=='C' : return 2
    elif c=='G' : return 3
    else :return -1

def DNAencoder(x):
    if len(x)!=2 : return -1;
    first = ATCG2num(x[0])
    second = ATCG2num(x[1])
    if first==-1 or second==-1 : return -1;
    return first*4+second;

decoded_data=genotype.applymap(DNAencoder);


## 2. 处理缺失的feature

In [4]:
features_missing=decoded_data.applymap(lambda x: x==-1 and 1 or 0).sum()
idx_features_missing = np.where(features_missing!=0)[0]

In [7]:
print 'missing feature: ',idx_features_missing
print decoded_data.columns[idx_features_missing]
print decoded_data.shape

missing feature:  [6069 8472]
Index([u'rs11573221', u'rs4252972'], dtype='object')
(1000, 9445)


In [8]:
probFeats = decoded_data.columns[idx_features_missing];
decoded_data[probFeats].describe()

,rs11573221,rs4252972
count,1000.0,1000.0
mean,-1.0,-1.0
std,0.0,0.0
min,-1.0,-1.0
25%,-1.0,-1.0
50%,-1.0,-1.0
75%,-1.0,-1.0
max,-1.0,-1.0


In [9]:
decoded_data.drop(decoded_data.columns[idx_features_missing],inplace=True,axis=1)
print decoded_data.shape

(1000, 9443)


## 3. OneHotEncode 编码

In [10]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
onehot_data = enc.fit_transform(decoded_data.as_matrix())

In [11]:
print decoded_data.shape
print onehot_data.shape

(1000, 9443)
(1000, 28329)


- 确认每三个对应原来的一个feature，这也和题目中的描述一致。

In [12]:
act=enc.active_features_;
fet=enc.feature_indices_;
for i in range(9443-1):
    if fet[i]<=act[3*i] and act[3*i+2]<fet[i+1]:
        pass
    else:
        print 'alert'

In [13]:
onehot_data = pd.DataFrame(onehot_data.toarray());
onehot_data.head()

,0,1,2,3,4,5,6,7,8,9,...,28319,28320,28321,28322,28323,28324,28325,28326,28327,28328
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## 4. 存储

In [14]:
store = pd.HDFStore('store.h5')
store['onehot_data']=onehot_data
store['data']=decoded_data
store.flush(fsync=True)
store.close()

In [15]:
print decoded_data.columns[8378]

rs7543405
